In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings 
warnings.filterwarnings("ignore")

# تحديد المسار الرئيسي للبيانات
directory = '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'

# الفئات وأسماؤها
categories = ["Bengin cases", "Malignant cases", "Normal cases"]

# حجم الصور الموحد
img_size = 224  # ResNet يفضل هذا الحجم

data = []
labels = []

for category in categories:
    path = os.path.join(directory, category)
    class_num = categories.index(category)
    
    for file in os.listdir(path):
        filepath = os.path.join(path, file)
        try:
            img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img_resized = cv2.resize(img, (img_size, img_size))
            data.append(img_resized)
            labels.append(class_num)
        except Exception as e:
            print(f"Error processing file {filepath}: {e}")

# تحويل القوائم إلى مصفوفات NumPy
data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0  # التطبيع إلى [0,1]
labels = np.array(labels)

# تقسيم البيانات
x_temp, x_test, y_temp, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_temp, y_temp, test_size=0.125, stratify=y_temp, random_state=42)

# التعامل مع الفئات غير المتوازنة باستخدام SMOTE
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train.reshape(-1, img_size * img_size), y_train)
x_train_resampled = x_train_resampled.reshape(-1, img_size, img_size, 1)

# تحويل الصور الرمادية إلى ثلاث قنوات
x_train_resampled = np.repeat(x_train_resampled, 3, axis=-1)
x_valid = np.repeat(x_valid, 3, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# إعداد الطبقة الأساسية ResNet152V2
base_model = ResNet152V2(
    include_top=False, 
    weights='imagenet', 
    input_shape=(224, 224, 3)
)

# البناء العلوي للنموذج
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # زيادة الإسقاط لمنع overfitting
output_layer = Dense(3, activation='softmax')(x)

# بناء النموذج النهائي
model = Model(inputs=base_model.input, outputs=output_layer)

# تثبيت الطبقات الأساسية للتدريب الأولي
base_model.trainable = False

# تجميع النموذج
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# استدعاءات التدريب
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_resnet_model.keras', monitor='val_loss', save_best_only=True)
]

# تدريب النموذج
history = model.fit(
    x_train_resampled, y_train_resampled,
    validation_data=(x_valid, y_valid),
    epochs=25,
    batch_size=32,
    callbacks=callbacks
)

# إلغاء تثبيت الطبقات الأساسية للتدريب الكامل
base_model.trainable = True

# إعادة تجميع النموذج بمعدل تعلم أقل
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# إعادة تدريب النموذج
history_fine_tune = model.fit(
    x_train_resampled, y_train_resampled,
    validation_data=(x_valid, y_valid),
    epochs=25,
    batch_size=32,
    callbacks=callbacks
)

# تقييم النموذج على بيانات الاختبار
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")


In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# توقع التصنيفات على بيانات الاختبار
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# حساب precision, recall, f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_classes, average=None)

# رسم القيم
categories = ["Bengin cases", "Malignant cases", "Normal cases"]
plt.figure(figsize=(10, 6))

plt.plot(categories, precision, label="Precision", marker='o')
plt.plot(categories, recall, label="Recall", marker='o')
plt.plot(categories, f1, label="F1-Score", marker='o')

plt.title("Precision, Recall, and F1-Score for each category")
plt.xlabel("Categories")
plt.ylabel("Score")
plt.legend()
plt.grid(True)
plt.show()

# عرض التقرير الكامل
print("Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=categories))

# حساب مصفوفة الارتباك
cm = confusion_matrix(y_test, y_pred_classes)

# رسم مصفوفة الارتباك
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=categories)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model

# وظيفة لإعداد Grad-CAM

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = Model(
        inputs=[model.input],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # تطبيع القيم بين 0 و 1 مع تحسين درجة التباين
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap)
    heatmap = np.power(heatmap, 0.8)  # زيادة التباين
    return heatmap

# وظيفة لتطبيق Grad-CAM على صورة

def apply_gradcam(image_path, label, model, last_conv_layer_name, input_size=(224, 224)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, input_size) / 255.0
    img_array = np.expand_dims(np.repeat(img_resized[..., np.newaxis], 3, axis=-1), axis=0)
    
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    heatmap_resized = cv2.resize(heatmap, input_size)
    heatmap_resized = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)
    heatmap_resized = cv2.cvtColor(heatmap_resized, cv2.COLOR_BGR2RGB)
    
    overlay = cv2.addWeighted(cv2.cvtColor((img_resized * 255).astype('uint8'), cv2.COLOR_GRAY2RGB), 0.6, heatmap_resized, 0.4, 0)
    
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Original Image ({label})")
    
    plt.subplot(1, 2, 2)
    plt.imshow(overlay)
    plt.title(f"Grad-CAM Heatmap ({label})")
    plt.show()

# تطبيق Grad-CAM على عينات من البيانات
last_conv_layer_name = 'conv5_block3_out'  # طبقة ResNet152V2 الأخيرة
sample_images = [
    '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/Bengin cases/Bengin case (1).jpg',
    '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/Malignant cases/Malignant case (106).jpg',
    '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset/Normal cases/Normal case (101).jpg'
]

labels = ["Bengin", "Malignant", "Normal"]
input_size = (224, 224)  # حجم المدخلات المطلوب لـ ResNet152V2

for image_path, label in zip(sample_images, labels):
    apply_gradcam(image_path, label, model, last_conv_layer_name, input_size=input_size)


In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy(train_acc, test_acc, epochs):
    """
    رسم بياني يعرض دقة التدريب ودقة الاختبار عبر الـ epochs.
    
    :param train_acc: دقة التدريب عبر الـ epochs
    :param test_acc: دقة الاختبار عبر الـ epochs
    :param epochs: عدد الـ epochs
    """
    plt.figure(figsize=(10, 6))
    
    # رسم دقة التدريب
    plt.plot(range(1, epochs + 1), train_acc, label='Training Accuracy', color='blue', linestyle='-', marker='o')
    
    # رسم دقة الاختبار
    plt.plot(range(1, epochs + 1), test_acc, label='Testing Accuracy', color='green', linestyle='--', marker='x')
    
    plt.title('Training and Testing Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# مثال للاستخدام:
train_acc = [0.5, 0.6, 0.7, 0.8, 0.85, 0.9]  # دقة التدريب عبر الـ epochs
test_acc = [0.45, 0.55, 0.65, 0.75, 0.8, 0.85]  # دقة الاختبار عبر الـ epochs
epochs = len(train_acc)  # عدد الـ epochs

# رسم الدقة
plot_accuracy(train_acc, test_acc, epochs)

In [ ]:
model.save("lung_model_web2.keras")

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import io

# Global variable to store the lung cancer model
lung_model = None

def load_model_if_needed():
    global lung_model
    if lung_model is None:
        try:
            # Load the lung cancer classifier model
            lung_model = tf.keras.models.load_model('/kaggle/working/best_resnet_model.keras')
            print("Lung cancer model loaded successfully.")
        except Exception as e:
            print(f"Error loading lung cancer model: {e}")
            raise

def preprocess_image(image):
    """
    Preprocess the input image:
    - Convert to grayscale (1 channel)
    - Resize to 256x256 (the input size expected by the model)
    - Normalize pixel values to the range [0, 1]
    - Expand dimensions to include the batch dimension
    """
    # Convert to grayscale
    image = image.convert('L')
    
    # Resize to 256x256
    image = image.resize((256, 256))
    
    # Convert the image to a numpy array and normalize
    img_array = np.array(image, dtype=np.float32) / 255.0
    
    # Add channel dimension (grayscale = 1 channel)
    img_array = np.expand_dims(img_array, axis=-1)
    
    # Add batch dimension: (1, 256, 256, 1)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_lung_cancer(image):
    """
    Given a PIL image, preprocess and predict the lung cancer class.
    Returns a dictionary containing:
      - 'prediction': Predicted class label.
      - 'confidence': Confidence for the predicted class.
      - 'prediction_vector': All prediction probabilities.
    """
    try:
        # Load the model if it hasn't been loaded yet
        load_model_if_needed()
        
        # Preprocess the image
        processed_image = preprocess_image(image)
        
        # Make the prediction
        prediction = lung_model.predict(processed_image, verbose=0)
        
        # Determine the predicted class and confidence
        predicted_class_index = np.argmax(prediction, axis=1)[0]
        confidence = float(prediction[0][predicted_class_index])
        
        # Map the predicted index to the corresponding class name.
        # Make sure the class order matches the one used during training.
        class_names = ['Normal', 'Malignant', 'Benign']
        prediction_class = class_names[predicted_class_index]
        
        return {
            "prediction": prediction_class,
            "confidence": confidence,
            "prediction_vector": prediction.tolist()  # Optional: full probability distribution
        }
    except Exception as e:
        print(f"Error in lung cancer prediction: {e}")
        raise

# Example usage:
if __name__ == '_main_':  # Correct
    # Replace 'path_to_image.jpg' with your image file path
    image_path = '/kaggle/input/ererrerere/Screenshot 2025-03-30 090551.png'
    try:
        image = Image.open(image_path)
        result = predict_lung_cancer(image)
        print("Prediction:", result["prediction"])
        print("Confidence:", result["confidence"])
    except Exception as e:
        print("An error occurred:", e)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import io

# Global variable to store the lung cancer model
lung_model = None

def load_model_if_needed():
    global lung_model
    if lung_model is None:
        try:
            # Load the lung cancer classifier model
            lung_model = tf.keras.models.load_model('/kaggle/working/best_resnet_model.keras')
            print("Lung cancer model loaded successfully.")
        except Exception as e:
            print(f"Error loading lung cancer model: {e}")
            raise

def preprocess_image(image):
    """
    Preprocess the input image:
    - Ensure the image is in RGB format (3 channels)
    - Resize to 224x224 (the input size expected by the model)
    - Normalize pixel values to the range [0, 1]
    - Expand dimensions to include the batch dimension
    """
    # Convert to RGB (ensure 3 channels)
    image = image.convert('RGB')
    
    # Resize to 224x224
    image = image.resize((224, 224))
    
    # Convert the image to a numpy array and normalize
    img_array = np.array(image, dtype=np.float32) / 255.0
    
    # Add batch dimension: (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_lung_cancer(image):
    """
    Given a PIL image, preprocess and predict the lung cancer class.
    Returns a dictionary containing:
      - 'prediction': Predicted class label.
      - 'confidence': Confidence for the predicted class.
      - 'prediction_vector': All prediction probabilities.
    """
    try:
        # Load the model if it hasn't been loaded yet
        load_model_if_needed()
        
        # Preprocess the image
        processed_image = preprocess_image(image)
        
        # Make the prediction
        prediction = lung_model.predict(processed_image, verbose=0)
        
        # Determine the predicted class and confidence
        predicted_class_index = np.argmax(prediction, axis=1)[0]
        confidence = float(prediction[0][predicted_class_index])
        
        # Map the predicted index to the corresponding class name.
        # Make sure the class order matches the one used during training.
        class_names = ['Normal', 'Malignant', 'Benign']
        prediction_class = class_names[predicted_class_index]
        
        return {
            "prediction": prediction_class,
            "confidence": confidence,
            "prediction_vector": prediction.tolist()  # Optional: full probability distribution
        }
    except Exception as e:
        print(f"Error in lung cancer prediction: {e}")
        raise

# Example usage:
if __name__ == '__main__':  # Correct
    # Replace 'path_to_image.jpg' with your image file path
    image_path = '/kaggle/input/chhvhdhd/Screenshot 2025-03-30 090704.png'
    try:
        image = Image.open(image_path)
        result = predict_lung_cancer(image)
        print("Prediction:", result["prediction"])
        print("Confidence:", result["confidence"])
    except Exception as e:
        print("An error occurred:", e)